# Installing Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import pytz
import berserk # A Python client for the Lichess API.
import os
import datetime

# Data Extraction

### Conection to *Berserk Client (API)*

In [2]:
token = os.environ.get('TOKEN')
session = berserk.TokenSession(token)
client = berserk.Client(session=session)
extract = client.games.export_by_player('YourKingIsInDanger',
            as_pgn=False, since=None, until=None, max=None, vs=None, rated=True, 
            perf_type=None, color=None, analysed=None, moves=True, tags=True, 
            evals=False, opening=True)
data = list(extract)

### Understanding the Extracted Data

In [3]:
type(data), type(data[0])

(list, dict)

In [4]:
data[0]

{'id': 'qnrlibcn',
 'rated': True,
 'variant': 'standard',
 'speed': 'blitz',
 'perf': 'blitz',
 'createdAt': datetime.datetime(2022, 6, 14, 13, 20, 41, 543000, tzinfo=datetime.timezone.utc),
 'lastMoveAt': datetime.datetime(2022, 6, 14, 13, 30, 45, 745000, tzinfo=datetime.timezone.utc),
 'status': 'resign',
 'players': {'white': {'user': {'name': 'YourKingIsInDanger',
    'id': 'yourkingisindanger'},
   'rating': 1659,
   'ratingDiff': -8},
  'black': {'user': {'name': 'Agustin_Perez', 'id': 'agustin_perez'},
   'rating': 1606,
   'ratingDiff': 7}},
 'winner': 'black',
 'opening': {'eco': 'C50',
  'name': 'Italian Game: Hungarian Defense',
  'ply': 6},
 'moves': 'e4 e5 Nf3 Nc6 Bc4 Be7 c3 d6 d4 exd4 cxd4 Bg4 h3 Bxf3 Qxf3 Nf6 Be3 O-O O-O a6 Nc3 b5 Bb3 Na5 Bd5 Nxd5 Nxd5 Nc4 b3 Nxe3 fxe3 Bg5 Rf2 c5 Raf1 f6 g3 Ra7 h4 Bh6 Qf5 Raf7 Kh2 Qd7 Qf3 cxd4 exd4 Qe6 Qf5 Qxf5 Rxf5 a5 g4 Bd2 Kg3 b4 Kf2 g6 Rf3 Kg7 Ke2 Bh6 e5 dxe5 dxe5 fxe5 Rxf7+ Rxf7 Rxf7+ Kxf7 Kd3 Bf8 Ke4 Bd6 Nb6 Ke6 Nc4 Bc7 h5 gxh5 gx

In [5]:
raw_df = pd.DataFrame(data)
raw_df.head(1)

,id,rated,variant,speed,perf,createdAt,lastMoveAt,status,players,winner,opening,moves,clock,tournament,initialFen
0,qnrlibcn,True,standard,blitz,blitz,2022-06-14 13:20:41.543000+00:00,2022-06-14 13:30:45.745000+00:00,resign,{'white': {'user': {'name': 'YourKingIsInDange...,black,"{'eco': 'C50', 'name': 'Italian Game: Hungaria...",e4 e5 Nf3 Nc6 Bc4 Be7 c3 d6 d4 exd4 cxd4 Bg4 h...,"{'initial': 300, 'increment': 0, 'totalTime': ...",NaN,NaN


In [6]:
raw_df.columns

Index(['id', 'rated', 'variant', 'speed', 'perf', 'createdAt', 'lastMoveAt',
       'status', 'players', 'winner', 'opening', 'moves', 'clock',
       'tournament', 'initialFen'],
      dtype='object')

# Data Cleaning

In our data we have\
1. id - *A unique game id of each game.*\
2. rated - *Is game rated or not: True/False*\
3. variant - *Game Variant: Standard, Chess960, From Position, CrazyHouse, etc.*\
4. speed - *Game Type: Rapid, Blitz, Bullet & classical*\
5. perf - *Similar of Game Type: Rapid, Blitz, Bullet & classical*\
6. createdAt - *Time at which game started: YYYY-MM-DD HH:MM:SS.ns+IST*\
7. lastMoveAt - *Time at which game ended: YYYY-MM-DD HH:MM:SS.ns+IST*\
8. status - *How the game ended: checkmate,resign,timeout,etc.*\
9. players - *Dictionary of players details Colorwise: Rating, Rating Change & id.*\
10. winner - *Color of the winner*\
11. opening - *A dictionary with Opening name, Eco & Ply for standard games.*\
12. moves -  *A string of the moves.*\
13. clock - *Time control of the game.*\
14. tournament - *Tournament id if the game is played in a tournament.*\
15. intialFen - *Opening for the game other than standard chess.*

---
I want to study my standard games as this is normal chess and it can only be analysed.\
• So I will select the variant as standard and rated as True.

#### Inspecting each field

###### 1. id

- This is a unique game id, which refers to a particular game on the Lichess Server.

In [7]:
# Renaming the column for better understanding
raw_df.rename(columns = {'id': 'Game ID'}, inplace = True)

###### 2. rated

In [8]:
raw_df['rated'].unique()

array([ True])

- It has only one value, so this column is no use of us

###### 3. variant 

In [9]:
raw_df['variant'].unique()

array(['standard', 'chess960', 'crazyhouse', 'fromPosition'], dtype=object)

- I only want standard variant so removing all others

In [10]:
raw_df = raw_df[raw_df['variant'] == 'standard'].copy()

- Now as all of the records have same variant that is standard, So this column is also no use of us

###### 4 & 5. speed & perf

In [11]:
raw_df['speed'].unique()

array(['blitz', 'rapid', 'bullet', 'classical'], dtype=object)

In [12]:
raw_df['perf'].unique()

array(['blitz', 'rapid', 'bullet', 'classical'], dtype=object)

- Both looks kind of same so let's check, if there are some data, where they differ in value

In [13]:
raw_df[raw_df['speed'] != raw_df['perf']]

,Game ID,rated,variant,speed,perf,createdAt,lastMoveAt,status,players,winner,opening,moves,clock,tournament,initialFen


- No, they are exactly same over all records. So, we can keep only 1 of them.

- Now, I only want Blitz, Rapid and Bullet so removing classical

In [14]:
raw_df = raw_df[raw_df['perf'] != 'classical'].copy()

In [15]:
# Formating
raw_df.rename(columns = {'perf': 'Game Type'}, inplace = True)
raw_df['Game Type'] = raw_df['Game Type'].str.capitalize()

###### 6 & 7. createdAt & lastMoveAt

In [16]:
raw_df[['createdAt','lastMoveAt']].dtypes

createdAt     datetime64[ns, UTC]
lastMoveAt    datetime64[ns, UTC]
dtype: object

- These are datetime objects but the timezone is UTC, we want Asia/Kolkata Timezone so,

In [17]:
raw_df['createdAt'] = raw_df['createdAt'].dt.tz_convert('Asia/Kolkata')
raw_df['lastMoveAt'] = raw_df['lastMoveAt'].dt.tz_convert('Asia/Kolkata')

In [18]:
raw_df[['createdAt','lastMoveAt']].head(1)

,createdAt,lastMoveAt
0,2022-06-14 18:50:41.543000+05:30,2022-06-14 19:00:45.745000+05:30


- They contain microseconds, but we don't need that. so,

In [19]:
raw_df['createdAt'] = pd.to_datetime(raw_df['createdAt'].dt.strftime('%Y-%m-%d %H:%M:%S'))
raw_df['lastMoveAt'] = pd.to_datetime(raw_df['lastMoveAt'].dt.strftime('%Y-%m-%d %H:%M:%S'))

In [20]:
# Creating Seperate Date and Time fields for Day Analysis.
raw_df['Start Time'] = raw_df['createdAt'].dt.time
raw_df['Start Date'] = (raw_df['createdAt'].dt.date).astype('datetime64')

raw_df['End Time'] = raw_df['lastMoveAt'].dt.time
raw_df['End Date'] = (raw_df['lastMoveAt'].dt.date).astype('datetime64')

In [21]:
raw_df.head(1)

,Game ID,rated,variant,speed,Game Type,createdAt,lastMoveAt,status,players,winner,opening,moves,clock,tournament,initialFen,Start Time,Start Date,End Time,End Date
0,qnrlibcn,True,standard,blitz,Blitz,2022-06-14 18:50:41,2022-06-14 19:00:45,resign,{'white': {'user': {'name': 'YourKingIsInDange...,black,"{'eco': 'C50', 'name': 'Italian Game: Hungaria...",e4 e5 Nf3 Nc6 Bc4 Be7 c3 d6 d4 exd4 cxd4 Bg4 h...,"{'initial': 300, 'increment': 0, 'totalTime': ...",NaN,NaN,18:50:41,2022-06-14,19:00:45,2022-06-14


###### 8. status

In [22]:
raw_df['status'].unique()

array(['resign', 'outoftime', 'timeout', 'mate', 'draw', 'stalemate'],
      dtype=object)

In [23]:
raw_df.rename(columns = {'status':'Status'}, inplace = True)

In [24]:
# Formating
raw_df['Status'].replace({'outoftime':'Out Of Time', 'timeout': 'Game Abandoned', 
                          'resign': 'Resign', 'mate': 'Checkmate', 'stalemate': 'Stalemate',
                         'draw': 'Draw'}, inplace = True)

In [25]:
raw_df['Status'].unique()

array(['Resign', 'Out Of Time', 'Game Abandoned', 'Checkmate', 'Draw',
       'Stalemate'], dtype=object)

#### 9. players

In [26]:
raw_df['players']

0       {'white': {'user': {'name': 'YourKingIsInDange...
1       {'white': {'user': {'name': 'Hanber', 'id': 'h...
2       {'white': {'user': {'name': 'YourKingIsInDange...
3       {'white': {'user': {'name': 'Gryym', 'id': 'gr...
4       {'white': {'user': {'name': 'YourKingIsInDange...
                              ...                        
1223    {'white': {'user': {'name': 'Osla', 'id': 'osl...
1224    {'white': {'user': {'name': 'Sanyi80', 'id': '...
1225    {'white': {'user': {'name': 'YourKingIsInDange...
1226    {'white': {'user': {'name': 'VeenaG', 'id': 'v...
1227    {'white': {'user': {'name': 'YourKingIsInDange...
Name: players, Length: 1212, dtype: object

In [27]:
raw_df['players'][0]

{'white': {'user': {'name': 'YourKingIsInDanger', 'id': 'yourkingisindanger'},
  'rating': 1659,
  'ratingDiff': -8},
 'black': {'user': {'name': 'Agustin_Perez', 'id': 'agustin_perez'},
  'rating': 1606,
  'ratingDiff': 7}}

- The entry is a dictionary, with alot of data, so creating new columns to save that data

In [28]:
raw_df['White_id'] = [x['white']['user']['id'] for x in raw_df['players']]
raw_df['White ELO'] = [x['white']['rating'] for x in raw_df['players']]
raw_df['White ELO Change'] = [x['white']['ratingDiff'] for x in raw_df['players']]

raw_df['Black_id'] = [x['black']['user']['id'] for x in raw_df['players']]
raw_df['Black ELO'] = [x['black']['rating'] for x in raw_df['players']]
raw_df['Black ELO Change'] = [x['black']['ratingDiff'] for x in raw_df['players']]

- The data is stored in geenral way on basis of color but i want data basis on player so,

In [29]:
raw_df['My ID'] = ''
raw_df['My Rating'] = ''
raw_df['My Rating Change'] = ''

raw_df['Opponent ID'] = ''
raw_df['Opponent Rating'] = ''
raw_df['Opponent Rating Change'] = ''
raw_df['My Color'] = ''

In [30]:
raw_df.head(1)

,Game ID,rated,variant,speed,Game Type,createdAt,lastMoveAt,Status,players,winner,...,Black_id,Black ELO,Black ELO Change,My ID,My Rating,My Rating Change,Opponent ID,Opponent Rating,Opponent Rating Change,My Color
0,qnrlibcn,True,standard,blitz,Blitz,2022-06-14 18:50:41,2022-06-14 19:00:45,Resign,{'white': {'user': {'name': 'YourKingIsInDange...,black,...,agustin_perez,1606,7,,,,,,,


In [31]:
raw_df.columns

Index(['Game ID', 'rated', 'variant', 'speed', 'Game Type', 'createdAt',
       'lastMoveAt', 'Status', 'players', 'winner', 'opening', 'moves',
       'clock', 'tournament', 'initialFen', 'Start Time', 'Start Date',
       'End Time', 'End Date', 'White_id', 'White ELO', 'White ELO Change',
       'Black_id', 'Black ELO', 'Black ELO Change', 'My ID', 'My Rating',
       'My Rating Change', 'Opponent ID', 'Opponent Rating',
       'Opponent Rating Change', 'My Color'],
      dtype='object')

- Index of the columns:\
• 'White_id' - 19         • 'White ELO' - 20         • 'White ELO Change' - 21\
• 'Black_id' - 22         • 'Black ELO' - 23         • 'Black ELO Change' - 24\
• 'My ID' - 25            • 'My Rating' - 26         • 'My Rating Change' - 27\
• 'Opponent ID' - 28      • 'Opponent Rating' - 29   • 'Opponent Rating Change' - 30\
• 'My Color' - 31

In [32]:
## Creating a function which will arrange the data on basis of players
def opp_my_distributor(checker, my, white, opp, black):
    for i in range(len(raw_df)):
        if raw_df.iloc[i,checker] == 'yourkingisindanger':
            raw_df.iloc[i,my] = raw_df.iloc[i,white]
            raw_df.iloc[i,opp] = raw_df.iloc[i,black]
            
        else:
            raw_df.iloc[i,my] = raw_df.iloc[i,black]
            raw_df.iloc[i,opp] = raw_df.iloc[i,white]

In [33]:
opp_my_distributor(19, 25, 19, 28, 22) ## ID
opp_my_distributor(19, 26, 20, 29, 23) ## Rating
opp_my_distributor(19, 27, 21, 30, 24) ## Rating Change

In [34]:
raw_df.loc[raw_df['White_id'] == 'yourkingisindanger', 'My Color'] = 'White'
raw_df.loc[raw_df['Black_id'] == 'yourkingisindanger', 'My Color'] = 'Black'

In [35]:
raw_df.head(1)

,Game ID,rated,variant,speed,Game Type,createdAt,lastMoveAt,Status,players,winner,...,Black_id,Black ELO,Black ELO Change,My ID,My Rating,My Rating Change,Opponent ID,Opponent Rating,Opponent Rating Change,My Color
0,qnrlibcn,True,standard,blitz,Blitz,2022-06-14 18:50:41,2022-06-14 19:00:45,Resign,{'white': {'user': {'name': 'YourKingIsInDange...,black,...,agustin_perez,1606,7,yourkingisindanger,1659,-8,agustin_perez,1606,7,White


###### 10. winner

In [36]:
raw_df['winner'].unique()

array(['black', 'white', nan], dtype=object)

- The nan values in winner denotes the matches where the result is Draw

In [37]:
# Making a Result with where i can record if i won, lose or draw.
raw_df.loc[raw_df['My Color'].str.lower() == raw_df['winner'], 'Result'] = "Won"
raw_df.loc[~(raw_df['My Color'].str.lower() == raw_df['winner']), 'Result'] = 'Lose'
raw_df.loc[raw_df['winner'].isna(), 'Result'] = 'Draw'

In [38]:
raw_df.head(1)

,Game ID,rated,variant,speed,Game Type,createdAt,lastMoveAt,Status,players,winner,...,Black ELO,Black ELO Change,My ID,My Rating,My Rating Change,Opponent ID,Opponent Rating,Opponent Rating Change,My Color,Result
0,qnrlibcn,True,standard,blitz,Blitz,2022-06-14 18:50:41,2022-06-14 19:00:45,Resign,{'white': {'user': {'name': 'YourKingIsInDange...,black,...,1606,7,yourkingisindanger,1659,-8,agustin_perez,1606,7,White,Lose


#### 11. opening

In [39]:
raw_df['opening']

0       {'eco': 'C50', 'name': 'Italian Game: Hungaria...
1       {'eco': 'C50', 'name': 'Italian Game: Anti-Fri...
2       {'eco': 'C42', 'name': 'Russian Game: Cozio At...
3       {'eco': 'E70', 'name': 'King's Indian Defense:...
4        {'eco': 'B00', 'name': 'Duras Gambit', 'ply': 2}
                              ...                        
1223    {'eco': 'C20', 'name': 'King's Pawn Game: Leon...
1224    {'eco': 'A40', 'name': 'Modern Defense', 'ply'...
1225    {'eco': 'C41', 'name': 'Philidor Defense', 'pl...
1226    {'eco': 'A00', 'name': 'Saragossa Opening', 'p...
1227    {'eco': 'B50', 'name': 'Sicilian Defense: Mode...
Name: opening, Length: 1212, dtype: object

- It is a Dictionary, so extracting the data using list comprehension

In [40]:
raw_df['Opening ECO'] = [ x['eco'] for x in raw_df['opening']]
raw_df['Opening'] = [ x['name'] for x in raw_df['opening']]
raw_df['Opening Ply'] = [ x['ply'] for x in raw_df['opening']]

In [41]:
raw_df['Opening']

0                         Italian Game: Hungarian Defense
1                  Italian Game: Anti-Fried Liver Defense
2                              Russian Game: Cozio Attack
3       King's Indian Defense: Accelerated Averbakh Va...
4                                            Duras Gambit
                              ...                        
1223                King's Pawn Game: Leonardis Variation
1224                                       Modern Defense
1225                                     Philidor Defense
1226                                    Saragossa Opening
1227                  Sicilian Defense: Modern Variations
Name: Opening, Length: 1212, dtype: object

- The Opening name consists of the Main name and the variation, I want them seperately So,

In [42]:
def seperator(x,y):
    try:
        l = x.split(':')
        if y == 0:
            return l[y]
        else:
            if len(l) == 1:
                return None
            else:
                return l[y]
            
    except:
        return None

In [43]:
raw_df['Opening Name'] = [seperator(x,0) for x in raw_df['Opening']]
raw_df['Opening Variation'] = [seperator(x,1) for x in raw_df['Opening']]

In [44]:
raw_df[['Opening','Opening Name', 'Opening Variation']].head()

,Opening,Opening Name,Opening Variation
0,Italian Game: Hungarian Defense,Italian Game,Hungarian Defense
1,Italian Game: Anti-Fried Liver Defense,Italian Game,Anti-Fried Liver Defense
2,Russian Game: Cozio Attack,Russian Game,Cozio Attack
3,King's Indian Defense: Accelerated Averbakh Va...,King's Indian Defense,Accelerated Averbakh Variation
4,Duras Gambit,Duras Gambit,None


###### 12. moves

In [45]:
raw_df['moves'][0]

'e4 e5 Nf3 Nc6 Bc4 Be7 c3 d6 d4 exd4 cxd4 Bg4 h3 Bxf3 Qxf3 Nf6 Be3 O-O O-O a6 Nc3 b5 Bb3 Na5 Bd5 Nxd5 Nxd5 Nc4 b3 Nxe3 fxe3 Bg5 Rf2 c5 Raf1 f6 g3 Ra7 h4 Bh6 Qf5 Raf7 Kh2 Qd7 Qf3 cxd4 exd4 Qe6 Qf5 Qxf5 Rxf5 a5 g4 Bd2 Kg3 b4 Kf2 g6 Rf3 Kg7 Ke2 Bh6 e5 dxe5 dxe5 fxe5 Rxf7+ Rxf7 Rxf7+ Kxf7 Kd3 Bf8 Ke4 Bd6 Nb6 Ke6 Nc4 Bc7 h5 gxh5 gxh5 Kf6 Kd5 Kf5 Ne3+ Kf4 Nc4 e4 Kd4 Kg5 Kxe4 Kxh5 Kd5 Kg4 Kc6 Bd8 Kb5 h5 Nxa5 Bxa5 Kxa5 h4 Kxb4 h3 a4 h2 a5 h1=Q a6 Qa8 a7 Qxa7 Kc4 Qb6 Kc3'

- a move in chess means 1 by each player so only e4 is half move, e4 e5 is a 1st move

In [46]:
## Calculating No. of Moves
raw_df['No. of Moves'] = [math.ceil(len(x.split())/2) for x in raw_df['moves']]

In [47]:
## Extracting first 4 Moves
raw_df['1.0 Move'] = [ x.split()[0]  for x in raw_df['moves']]
raw_df['1.1 Move'] = [ x.split()[1]  for x in raw_df['moves']]
raw_df['2.0 Move'] = [ x.split()[2]  for x in raw_df['moves']]
raw_df['2.1 Move'] = [ x.split()[3]  for x in raw_df['moves']]

In [48]:
## Alloting the first 4 moves to players
raw_df.loc[raw_df['My Color'] == 'White', 'M.1'] = raw_df[raw_df['My Color'] == 'White']['1.0 Move']
raw_df.loc[raw_df['My Color'] == 'White', 'M.2'] = raw_df[raw_df['My Color'] == 'White']['2.0 Move']
raw_df.loc[raw_df['My Color'] == 'Black', 'M.1'] = raw_df[raw_df['My Color'] == 'Black']['1.1 Move']
raw_df.loc[raw_df['My Color'] == 'Black', 'M.2'] = raw_df[raw_df['My Color'] == 'Black']['2.1 Move']

raw_df.loc[raw_df['My Color'] == 'White', 'O.1'] = raw_df[raw_df['My Color'] == 'White']['1.1 Move']
raw_df.loc[raw_df['My Color'] == 'White', 'O.2'] = raw_df[raw_df['My Color'] == 'White']['2.1 Move']
raw_df.loc[raw_df['My Color'] == 'Black', 'O.1'] = raw_df[raw_df['My Color'] == 'Black']['1.0 Move']
raw_df.loc[raw_df['My Color'] == 'Black', 'O.2'] = raw_df[raw_df['My Color'] == 'Black']['2.0 Move']

In [49]:
raw_df.head(1)

,Game ID,rated,variant,speed,Game Type,createdAt,lastMoveAt,Status,players,winner,...,Opening Variation,No. of Moves,1.0 Move,1.1 Move,2.0 Move,2.1 Move,M.1,M.2,O.1,O.2
0,qnrlibcn,True,standard,blitz,Blitz,2022-06-14 18:50:41,2022-06-14 19:00:45,Resign,{'white': {'user': {'name': 'YourKingIsInDange...,black,...,Hungarian Defense,58,e4,e5,Nf3,Nc6,e4,Nf3,e5,Nc6


#### 13. clock

In [50]:
raw_df['clock']

0       {'initial': 300, 'increment': 0, 'totalTime': ...
1       {'initial': 300, 'increment': 0, 'totalTime': ...
2       {'initial': 300, 'increment': 0, 'totalTime': ...
3       {'initial': 300, 'increment': 0, 'totalTime': ...
4       {'initial': 300, 'increment': 0, 'totalTime': ...
                              ...                        
1223    {'initial': 180, 'increment': 0, 'totalTime': ...
1224    {'initial': 180, 'increment': 0, 'totalTime': ...
1225    {'initial': 180, 'increment': 0, 'totalTime': ...
1226    {'initial': 180, 'increment': 0, 'totalTime': ...
1227    {'initial': 180, 'increment': 0, 'totalTime': ...
Name: clock, Length: 1212, dtype: object

In [51]:
raw_df['clock'][0]

{'initial': 300, 'increment': 0, 'totalTime': 300}

- Fomating it to the format: 5 Min + 0 Sec

In [52]:
raw_df['Time Control'] = ['{} Min + {} Sec'.format((x['initial']//60), x['increment']) for x in raw_df['clock'] ]

In [53]:
raw_df.head(1)

,Game ID,rated,variant,speed,Game Type,createdAt,lastMoveAt,Status,players,winner,...,No. of Moves,1.0 Move,1.1 Move,2.0 Move,2.1 Move,M.1,M.2,O.1,O.2,Time Control
0,qnrlibcn,True,standard,blitz,Blitz,2022-06-14 18:50:41,2022-06-14 19:00:45,Resign,{'white': {'user': {'name': 'YourKingIsInDange...,black,...,58,e4,e5,Nf3,Nc6,e4,Nf3,e5,Nc6,5 Min + 0 Sec


###### 14. tournament

In [54]:
raw_df['tournament'].unique()

array([nan, '7qLQnyrI', 'Kbq6hs06', '1NUtWJEq', 'Z1JJNSIj', 'UZu3qGKo',
       'ZYV0J6y4', 'r6oTBiYG', 'IPqQZGlp', 'lkwODJck', 'DBEYi5oB',
       'llRxupSK', 'VA9FDXDc', '90ubk1Kc', 'VCDjG9zz', 'K52V3lsw',
       'fkaxbE7M', 'bSUXiKLg', 'nJIzx6gv', '2VuIsji8', 'Ah8UZJ0q',
       'KEGrsI1X', 'FDpULmSl', 'iNaKmb1Q', 'tMjwUbJy', 'S0b6Su1w',
       'j4gzLrhD', 'DCqdT9XM', 'KiqK3dw9', 'IS972gpN', 'auAEb8Ea',
       'FcsYKtz3', 'M8Y2wNS0', 'uYI0yFDk', 'eeez5QYn', 'sCVzW92g',
       '0mScIt6f', 'DKBjiyqb', 'i1AeUK1a', 'iL1zBh0t', 'snbdoMJL',
       'iPdbatnK', 'r5FgKQae', 'PhPhROoO', 'qkib6R1a', 'HgCyVikw',
       'YYVetI00', 'CXHLrLsZ', 'kARcOy6S', 'xxu91IZP', 'Cix3nOtA',
       '6gzzLiBH', 'tM1o9SZB', 'AWI8SOSG', 'ln53skXN', 'fY3JY2yM',
       'hRoPjUvq', 'pSWL0meH', 'HyA1mM4y'], dtype=object)

So, if the game is played in a tournament it has a tournamnt id Where else there is None, so we'll make two columns,
1. Tournament Game - bool type
2. Tournament Id - str type

In [55]:
raw_df.loc[raw_df['tournament'].isna(), 'Tournament Game'] = False
raw_df.loc[~(raw_df['tournament'].isna()), 'Tournament Game'] = True

In [56]:
raw_df.rename(columns = {'tournament':'Tournament ID'}, inplace = True)

###### 15. initialFen

In [57]:
raw_df['initialFen'].unique()

array([nan], dtype=object)

It has nothing

In [58]:
## Taking our Usefull Variables in a new Dataframe

In [59]:
raw_df.columns

Index(['Game ID', 'rated', 'variant', 'speed', 'Game Type', 'createdAt',
       'lastMoveAt', 'Status', 'players', 'winner', 'opening', 'moves',
       'clock', 'Tournament ID', 'initialFen', 'Start Time', 'Start Date',
       'End Time', 'End Date', 'White_id', 'White ELO', 'White ELO Change',
       'Black_id', 'Black ELO', 'Black ELO Change', 'My ID', 'My Rating',
       'My Rating Change', 'Opponent ID', 'Opponent Rating',
       'Opponent Rating Change', 'My Color', 'Result', 'Opening ECO',
       'Opening', 'Opening Ply', 'Opening Name', 'Opening Variation',
       'No. of Moves', '1.0 Move', '1.1 Move', '2.0 Move', '2.1 Move', 'M.1',
       'M.2', 'O.1', 'O.2', 'Time Control', 'Tournament Game'],
      dtype='object')

In [60]:
Chess_df = raw_df[['Game ID', 'Start Date','Start Time', 'End Date', 'End Time', 'Game Type',
        'Time Control', 'Tournament Game', 'Tournament ID', 'createdAt', 'lastMoveAt',
        'My ID', 'My Rating', 'My Rating Change', 'My Color',
        'Opponent ID', 'Opponent Rating', 'Opponent Rating Change', 'Status',  'Result',
        'Opening ECO', 'Opening Ply', 'Opening', 'Opening Variation', 
        'No. of Moves', '1.0 Move', '1.1 Move', '2.0 Move', '2.1 Move', 
        'M.1', 'M.2', 'O.1', 'O.2']].copy()

In [61]:
Chess_df.head()

,Game ID,Start Date,Start Time,End Date,End Time,Game Type,Time Control,Tournament Game,Tournament ID,createdAt,...,Opening Variation,No. of Moves,1.0 Move,1.1 Move,2.0 Move,2.1 Move,M.1,M.2,O.1,O.2
0,qnrlibcn,2022-06-14,18:50:41,2022-06-14,19:00:45,Blitz,5 Min + 0 Sec,False,NaN,2022-06-14 18:50:41,...,Hungarian Defense,58,e4,e5,Nf3,Nc6,e4,Nf3,e5,Nc6
1,CwbVPlQf,2022-06-12,14:46:40,2022-06-12,14:52:41,Blitz,5 Min + 0 Sec,False,NaN,2022-06-12 14:46:40,...,Anti-Fried Liver Defense,33,e4,e5,Bc4,Nc6,e5,Nc6,e4,Bc4
2,v4Jyr7UC,2022-06-12,14:39:05,2022-06-12,14:46:28,Blitz,5 Min + 0 Sec,False,NaN,2022-06-12 14:39:05,...,Cozio Attack,26,e4,e5,Nf3,Nf6,e4,Nf3,e5,Nf6
3,Fs56klNV,2022-06-11,21:07:25,2022-06-11,21:16:46,Blitz,5 Min + 0 Sec,False,NaN,2022-06-11 21:07:25,...,Accelerated Averbakh Variation,36,d4,g6,e4,Bg7,g6,Bg7,d4,e4
4,jv5yaYp9,2022-06-10,02:19:50,2022-06-10,02:28:04,Blitz,5 Min + 0 Sec,False,NaN,2022-06-10 02:19:50,...,None,27,e4,f5,exf5,Kf7,e4,exf5,f5,Kf7


# Creating Calculated Fields

###### New Rating

In [62]:
Chess_df['My New Rating'] = Chess_df['My Rating'] + Chess_df['My Rating Change']

In [63]:
Chess_df.head(2)

,Game ID,Start Date,Start Time,End Date,End Time,Game Type,Time Control,Tournament Game,Tournament ID,createdAt,...,No. of Moves,1.0 Move,1.1 Move,2.0 Move,2.1 Move,M.1,M.2,O.1,O.2,My New Rating
0,qnrlibcn,2022-06-14,18:50:41,2022-06-14,19:00:45,Blitz,5 Min + 0 Sec,False,NaN,2022-06-14 18:50:41,...,58,e4,e5,Nf3,Nc6,e4,Nf3,e5,Nc6,1651
1,CwbVPlQf,2022-06-12,14:46:40,2022-06-12,14:52:41,Blitz,5 Min + 0 Sec,False,NaN,2022-06-12 14:46:40,...,33,e4,e5,Bc4,Nc6,e5,Nc6,e4,Bc4,1659


##### Time Interval

In [64]:
for i in range(24):
    Chess_df.loc[(datetime.time(i,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(i,59,59)), 'Time Interval'] = \
            '{} {} - {} {}'.format(12 if i == 0 else i if i < 13 else i-12, 'AM' if i < 12 else 'PM', 
                                   i+1 if i+1 < 13 else i+1 - 12, 'AM' if i==23 else 'AM' if i+1 < 12 else 'PM')

###### Day Time

• 4AM - 7AM : Early Morning\
• 7AM - 9AM : Morning\
• 9AM - 12AM : Late Morning\
• 12AM - 3AM : Afternooon\
• 3AM - 5AM : Late Afternoon\
• 5AM - 7AM : Early Evening\
• 7AM - 9AM : Evening\
• 9AM - 11AM : Late Evening\
• 11AM - 2AM : Night\
• 2AM - 4AM : Late night

In [65]:
Chess_df.loc[(datetime.time(0,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(1,59,59)), 'Day Time'] = 'Night'

Chess_df.loc[(datetime.time(2,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(3,59,59)), 'Day Time'] = 'Late Night'

Chess_df.loc[(datetime.time(4,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(6,59,59)), 'Day Time'] = 'Early Morning'

Chess_df.loc[(datetime.time(7,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(8,59,59)), 'Day Time'] = 'Morning'

Chess_df.loc[(datetime.time(9,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(11,59,59)), 'Day Time'] = 'Late Morning'

Chess_df.loc[(datetime.time(12,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(14,59,59)), 'Day Time'] = 'Afternoon'

Chess_df.loc[(datetime.time(15,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(16,59,59)), 'Day Time'] = 'Late Afternoon'

Chess_df.loc[(datetime.time(17,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(19,59,59)), 'Day Time'] = 'Early Evening'

Chess_df.loc[(datetime.time(19,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(20,59,59)), 'Day Time'] = 'Evening'

Chess_df.loc[(datetime.time(21,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(22,59,59)), 'Day Time'] = 'Late Evening'

Chess_df.loc[(datetime.time(23,0,0) < Chess_df['Start Time']) & \
        (Chess_df['Start Time'] < datetime.time(23,59,59)), 'Day Time'] = 'Night'

##### Game Stage

0-15 Moves - Opening\
16-49 Moves - MidGame\
50+ Moves - EndGame


In [66]:
Chess_df.loc[Chess_df['No. of Moves'] <= 15, 'Game Stage'] = 'Opening'
Chess_df.loc[(Chess_df['No. of Moves'] >= 16) &\
             (Chess_df['No. of Moves'] <= 49), 'Game Stage'] = 'Middlegame'
Chess_df.loc[Chess_df['No. of Moves'] >= 50, 'Game Stage'] = 'Endgame'

###### Is Rating Difference More Than Me

In [67]:
Chess_df.loc[(Chess_df['Opponent Rating'] - Chess_df['My Rating']) > 0, 
             'Rating Difference More Than Me'] = True

Chess_df.loc[(Chess_df['Opponent Rating'] - Chess_df['My Rating']) <= 0, 
             'Rating Difference More Than Me'] = False

# Saving The File

In [68]:
Chess_df.to_csv('Chess_df.csv')

In [69]:
Chess_df.head(2)

,Game ID,Start Date,Start Time,End Date,End Time,Game Type,Time Control,Tournament Game,Tournament ID,createdAt,...,2.1 Move,M.1,M.2,O.1,O.2,My New Rating,Time Interval,Day Time,Game Stage,Rating Difference More Than Me
0,qnrlibcn,2022-06-14,18:50:41,2022-06-14,19:00:45,Blitz,5 Min + 0 Sec,False,NaN,2022-06-14 18:50:41,...,Nc6,e4,Nf3,e5,Nc6,1651,6 PM - 7 PM,Early Evening,Endgame,False
1,CwbVPlQf,2022-06-12,14:46:40,2022-06-12,14:52:41,Blitz,5 Min + 0 Sec,False,NaN,2022-06-12 14:46:40,...,Nc6,e5,Nc6,e4,Bc4,1659,2 PM - 3 PM,Afternoon,Middlegame,False
